https://www.kaggle.com/code/arezoodahesh/customer-churn-with-oversampling-techniques#Import-Libraries

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

In [43]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, KFold

from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, RandomOverSampler

from scipy import stats
import joblib

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [4]:
from funcs.plots.confusion_matrix import plot_confusion_matrix
from funcs.plots.roc_curves import plot_roc_curves, plot_roc_area_curve
from funcs.plots.tpr_fpr_thresholds import plot_tpr_fpr_thresholds
from funcs.plots.distribution import plot_violin_binary_dist, plots_kde

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [25]:
df = pd.read_csv("F:/Data/datas/WA_Fn-UseC_-Telco-Customer-Churn.csv", index_col=0)
df['Churn'] = df['Churn'].map({'Yes':1, 'No':0})

In [14]:
NUM_COLS = ["tenure", "MonthlyCharges", "TotalCharges"]

## Data Preparation

### Data Cleaning

#### Fill Missing Values

- 0 tenure value causes the blank value in the total charges column and makes that column detected automatically as object data type
- We replace the blank string values with 0.0 float number

In [32]:
df[df['tenure'] == 0][['tenure', 'TotalCharges']]

,tenure,TotalCharges
customerID,,
4472-LVYGI,0,0.0
3115-CZMZD,0,0.0
5709-LVOEQ,0,0.0
4367-NUYAO,0,0.0
1371-DWPAZ,0,0.0
7644-OMVMY,0,0.0
3213-VVOLG,0,0.0
2520-SGTTA,0,0.0
2923-ARZLG,0,0.0


In [34]:
df.loc[df['tenure'] == 0, 'TotalCharges'] = 0.0
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print("Missing Values:", df['TotalCharges'].isna().sum())

Missing Values: 0


#### Handling Outliers (Optional)

In [36]:
def replace_outliers(df, columns):
    df = df.copy()
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3-Q1
        
        top_cap = Q3 + (IQR * 1.5)
        bottom_cap = Q1 - (IQR * 1.5)
        
        if df[col].max() > top_cap:
            print("above bottom cap:", df[df[col] > top_cap].shape[0])            
            df[col] = df[col].apply(lambda x: top_cap if x > top_cap else x)
        if df[col].min() < bottom_cap:
            print("below bottom cap:", df[df[col] < top_cap].shape[0])
            df[col] = df[col].apply(lambda x: bottom_cap if x < bottom_cap else x)
                
    return df

In [37]:
def replace_outliers_by_group(df, columns, col_group):
    df = df.copy()
    groups = df[col_group].unique()
    
    for col in columns:
        df[col] = df[col].astype(float)
    
    for group in groups:
        datas = df[df[col_group] == group]
        for col in columns:
            data = datas[col]
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3-Q1

            top_cap = Q3 + (IQR * 1.5)
            bottom_cap = Q1 - (IQR * 1.5)

            if data.max() > top_cap:                
                idx_above_cap = data[data > top_cap].index
                print(f"above top cap ({col} | {group}):", len(idx_above_cap))
                df.loc[idx_above_cap, col] = top_cap
            if df[col].min() < bottom_cap:
                idx_below_cap = data[data < bottom_cap].index
                print(f"below bottom cap ({col}):", len(idx_below_cap))
                df.loc[idx_below_cap, col] = idx_below_cap
                
    return df

##### Without Grouping

No outliers were detected when we didn't group them by "Churn" label

In [ ]:
plots_kde(df, ['tenure', 'TotalCharges', 'MonthlyCharges'], 1)

In [ ]:
df = replace_outliers(df, ['tenure', 'TotalCharges', 'MonthlyCharges'])

##### Grouping by Churn

Some outliers were detected if we group them by "Churn"

In [ ]:
plot_violin_binary_dist(df, 'tenure', 'Churn')

In [ ]:
plot_violin_binary_dist(df, 'TotalCharges', 'Churn')

In [ ]:
plot_violin_binary_dist(df, 'MonthlyCharges', 'Churn')

In [38]:
df = replace_outliers_by_group(df, ['tenure', 'TotalCharges', 'MonthlyCharges'], 'Churn')

above top cap (tenure | 1): 23
above top cap (TotalCharges | 1): 109


### Features Encoding

#### Nominal Data into Binary

We convert our nominal data into binary values.

In [ ]:
for col in df.select_dtypes('object').columns:
    print(f"{col}: \n{df[col].unique()}\n")

In [39]:
df['gender'] = df['gender'].map({'Male':1, 'Female':0})
df['Partner'] = df['Partner'].map({'Yes':1, 'No':0})
df['Dependents'] = df['Dependents'].map({'Yes':1, 'No':0})
df['PhoneService'] = df['PhoneService'].map({'Yes':1, 'No':0})
df['PaperlessBilling'] = df['PaperlessBilling'].map({'Yes':1, 'No':0})

In [ ]:
df.describe(include='object')

#### Ordinal Data into Stratificiation encoding

We considered ordinal data has a stratification values

In [41]:
def cross_validated_target_encoding(df, categorical_col, target_col, n_splits=5, smoothing=1.0):
    df = df.copy()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    target_mean_global = df[target_col].mean()

    df['encoded'] = np.nan

    for train_idx, val_idx in kf.split(df):
        train, val = df.iloc[train_idx], df.iloc[val_idx]
        category_means = train.groupby(categorical_col)[target_col].mean()
        
        # Smoothing
        counts = train[categorical_col].value_counts()
        smoothed_means = (category_means * counts + target_mean_global * smoothing) / (counts + smoothing)
        
        df.loc[val_idx, 'encoded'] = val[categorical_col].map(smoothed_means)
    
    df[categorical_col] = df['encoded'].fillna(target_mean_global)
    df.drop('encoded', axis=1, inplace=True)

    return df

In [44]:
data = pd.DataFrame({
    'category_feature': ['A', 'B', 'A', 'C', 'B', 'A'],
    'numeric_feature': [1, 2, 3, 4, 5, 6],
    'target': [10, 20, 30, 40, 50, 60]
})

encoded_data = cross_validated_target_encoding(data, 'category_feature', 'target')
print(encoded_data)

   category_feature  numeric_feature  target
0         41.666667                1      10
1         42.500000                2      20
2         35.000000                3      30
3         35.000000                4      40
4         27.500000                5      50
5         25.000000                6      60


In [45]:
encoded_data

,category_feature,numeric_feature,target
0,41.666667,1,10
1,42.500000,2,20
2,35.000000,3,30
3,35.000000,4,40
4,27.500000,5,50
5,25.000000,6,60


In [ ]:
def target_encoder(df, y):
    df = df.copy()
    mapping = {}
    
    # Encode target variable if it's not numeric
    if df[y].dtype == 'object' or not pd.api.types.is_numeric_dtype(df[y]):
        df[y] = LabelEncoder().fit_transform(df[y])
    
    for col in df.select_dtypes(include='object').columns:
        # Calculate the mean target value for each category
        mean_encoded = df.groupby(col)[y].mean().to_dict()
        mapping[col] = mean_encoded
        
        # Replace each category with its corresponding mean target value
        df[col] = df[col].map(mean_encoded)
    
    return df, mapping

In [ ]:
df, mapping = target_encoder(df, 'Churn')

In [40]:
df.select_dtypes(exclude='number').nunique()

MultipleLines       3
InternetService     3
OnlineSecurity      3
OnlineBackup        3
DeviceProtection    3
TechSupport         3
StreamingTV         3
StreamingMovies     3
Contract            3
PaymentMethod       4
dtype: int64

### Data Preprocessing and Processing

#### Data Scaling (Change the range of values)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
df[NUM_COLS].hist(figsize=(12, 2), bins=100, layout=(1, 3));

In [ ]:
ss = StandardScaler()
df.loc[:, NUM_COLS] = ss.fit_transform(df[NUM_COLS])

In [ ]:
df[NUM_COLS].hist(figsize=(12, 2), bins=100, layout=(1, 3));

#### Data Transformation (Change the shape of distribution)

In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
print("skewness")
print('yeo-johnson: ', pd.DataFrame(PowerTransformer(method='yeo-johnson').fit_transform(df[NUM_COLS])).skew().abs().sum())
print('box-cox: ', pd.DataFrame(PowerTransformer(method='box-cox').fit_transform(df[NUM_COLS])).skew().abs().sum())
print('sqrt: ', np.sqrt(df[NUM_COLS]).skew().abs().sum())
print('cbrt: ', np.cbrt(df[NUM_COLS]).skew().abs().sum())
print('log: ', np.log(df[NUM_COLS]).skew().abs().sum())
print('log1p: ', np.log1p(df[NUM_COLS]).skew().abs().sum())

In [ ]:
df[NUM_COLS] = PowerTransformer(method='yeo-johnson').fit_transform(df[NUM_COLS])

In [ ]:
df[NUM_COLS].hist(figsize=(12, 2), bins=100, layout=(1, 3));

## Data Modeling

### Data Splitting

In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Handling Imbalanced Data Label

- We do oversampling technique that increases the number of samples in the minority class
- We leave the test data in its original distribution to accurately evaluate model performance.
- The test set should reflect real-world scenarios where the class distribution is naturally imbalanced.

#### SMOTE

In [ ]:
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote_resampled, y_train_smote_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
y_train_smote_resampled.value_counts()

### Feature Selection

In [ ]:
def checkVIF(df, scale=True):    
    index = df.columns.tolist()
    
    if scale:
        df = pd.DataFrame(StandardScaler().fit_transform(df))

    vif = pd.Series([variance_inflation_factor(df.values, i) for i in range(df.shape[1])], index=index)
    
    return pd.DataFrame(vif, index=index, columns=['vif']).sort_values(by='vif', ascending=False)

In [ ]:
checkVIF(X_train_smote_resampled, True)

### Models Fitting

#### Llogistic Regression

In [ ]:
model_lg = LogisticRegression(max_iter=100).fit(X_train, y_train)
y_pred_lg = model_lg.predict(X_test)
y_score_lg = model_lg.predict_proba(X_test)[:,1]

In [ ]:
print("train score:", model_lg.score(X_train, y_train))
print("test score:", model_lg.score(X_test, y_test))

In [ ]:
model_lg2 = LogisticRegression(max_iter=100).fit(X_train_smote_resampled, y_train_smote_resampled)
print("train score:", model_lg2.score(X_train_smote_resampled, y_train_smote_resampled))
print("test score:", model_lg2.score(X_test, y_test))

#### Random Forest

In [ ]:
model_rf = RandomForestClassifier(max_depth=8).fit(X_train, y_train)
print("train score:", model_rf.score(X_train, y_train))
print("test score:", model_rf.score(X_test, y_test))

In [ ]:
model_rf2 = RandomForestClassifier(max_depth=7).fit(X_train_smote_resampled, y_train_smote_resampled)
print("train score:", model_rf2.score(X_train_smote_resampled, y_train_smote_resampled))
print("test score:", model_rf2.score(X_test, y_test))

#### XGB Classifier

In [ ]:
model_xgb_clf = XGBClassifier(eval_metric='aucpr', early_stopping_rounds=10, )
model_xgb_clf.fit(
    X_train,
    y_train, 
    verbose=False,    
    eval_set=[(X_test, y_test)]
)

In [ ]:
print("train score:", model_xgb_clf.score(X_train, y_train))
print("test score:", model_xgb_clf.score(X_test, y_test))

In [ ]:
model_xgb_clf2 = XGBClassifier(eval_metric='aucpr', early_stopping_rounds=10)
model_xgb_clf2.fit(
    X_train_smote_resampled, 
    y_train_smote_resampled, 
    verbose=False,
    eval_set=[(X_test, y_test)]
)

In [ ]:
print("train score:", model_xgb_clf2.score(X_train_smote_resampled, y_train_smote_resampled))
print("test score:", model_xgb_clf2.score(X_test, y_test))

## Model Optimization

## Model Evaluation

### Classification Report

In [ ]:
print(classification_report(y_pred_lg, y_test, output_dict=False))

The classification report indicates that our model:

- Overall Accuracy: Achieves an accuracy of 81% (f1-score).
- Performance on Non-Churn Customers:
    - Accuracy: 88% f1-score.
    - Precision: 90% (higher than recall), indicating that the model tends to assume customers are loyal.
    - Recall: 85%, which, along with the higher precision, suggests an imbalance in the dataset where non-churn cases are more prevalent.
- Performance on Churn Customers:
    - Accuracy: 62% f1-score, indicating weaker performance.
    - Precision: 57% (lower than recall), suggesting the model is cautious in predicting churn, leading to fewer false positives.
    - Recall: 62%, which shows the model identifies more actual churn cases but at the cost of lower precision.
- Overall, the model shows good performance in predicting non-churn customers but struggles with accurately identifying churn customers, highlighting areas for potential improvement

In [ ]:
pd.DataFrame(classification_report(y_pred_lg, y_test, output_dict=True)).T.iloc[:,:-1]

### Confusion Matrix

In [ ]:
fig_cm = plot_confusion_matrix(confusion_matrix(y_test, y_pred_lg))
fig_cm

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_lg).ravel()

In [ ]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred_lg), 
    index=['Actual Negative', 'Actual Positive'], 
    columns=['Predicted Negative', 'Predicted Positive']
)

Our confusion matrix shows the following:

- True Negative (1159), the model predicted negative and the actual was also negative.
- False Positive (123), the model predicted positive but the actual was negative.
- True Positive (273), the model predicted positive and the actual was also positive.
- False Negative (206), the model predicted negative but the actual was positive.

### TPR (True Positive Rate) and (False Positive Rate) Thresholds

- True Positive Rate (also known as recall or sensitivity) measures the proportion of true positive cases correctly identified by the model among all actual positive cases. It is calculated as the ratio of true positives to the sum of true positives and false negatives.
- False Positive Rate measures the proportion of false positive cases incorrectly identified as positive by the model among all actual negative cases. It is calculated as the ratio of false positives to the sum of false positives and true negatives.

In [ ]:
fig_tpr_fpr = plot_tpr_fpr_thresholds(lg_model, X_test, y_test)
fig_tpr_fpr.update_layout(width=None)

### ROC Curves

In [ ]:
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score

In [ ]:
def calculate_auc(fpr, tpr):
    """
    Calculate the Area Under the Curve (AUC) using the trapezoidal rule.
    
    Parameters:
    fpr (list): List of False Positive Rates (FPR).
    tpr (list): List of True Positive Rates (TPR).

    Returns:
    float: The AUC value.
    """
    # Ensure the lists are sorted in ascending order of FPR
    sorted_indices = sorted(range(len(fpr)), key=lambda i: fpr[i])
    fpr = [fpr[i] for i in sorted_indices]
    tpr = [tpr[i] for i in sorted_indices]

    # Initialize AUC
    auc = 0.0

    # Apply the trapezoidal rule
    for i in range(1, len(fpr)):
        auc += (fpr[i] - fpr[i-1]) * (tpr[i] + tpr[i-1]) / 2

    return auc

In [ ]:
accuracy_score(y_test, y_pred_custom[:, 0])

In [ ]:
accuracy_score(y_test, y_pred_lg)

In [ ]:
y_score = lg_model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])

df_threshold = pd.DataFrame({"fpr": fpr, "tpr": tpr, "thresholds": thresholds})
df_threshold = df_threshold.sort_values('thresholds').reset_index(drop=True)
df_threshold['different'] = df_threshold['tpr'] - df_threshold['fpr']

In [ ]:
df_threshold.sort_values('different')

In [ ]:
roc_auc_score(y_test, y_score[:, 1])

In [ ]:
fig_roc = plot_roc_curves(lg_model, X_test, y_test)
fig_roc.update_layout(width=None)

### Area AUC (Area Under Curve)

In [ ]:
fig_area_auc = plot_roc_area_curve(lg_model, X_test, y_test, 1)
fig_area_auc.update_layout(width=None)